In [1]:
from main import download_data
CRYPTOS = ['BTC', 'ETH', 'EOS', 'LTC', 'XRP', 'BCH', 'ADA', 'XLM', 'CVC']

In [2]:
download_data(CRYPTOS, currency='USD')

Querying for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results for BTC
Still fetching results

In [3]:
from datetime import datetime, timedelta
from typing import List, Tuple
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [4]:
spark = SparkSession.builder.appName("Crypto Data").getOrCreate()
df = spark.read.csv("/home/jovyan/data/data_raw.csv", inferSchema=True, encoding='utf-8', header=True)
df.cache()
df.printSchema()

root
 |-- close: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- open: double (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- volumefrom: double (nullable = true)
 |-- volumeto: double (nullable = true)
 |-- symbol: string (nullable = true)



In [5]:
df.show(3, truncate=False)

+------+------+------+------+-------------------+----------+--------+------+
|close |high  |low   |open  |time               |volumefrom|volumeto|symbol|
+------+------+------+------+-------------------+----------+--------+------+
|0.0616|0.0616|0.0616|0.0616|2010-09-05 22:00:00|0.0       |0.0     |BTC   |
|0.0616|0.0616|0.0616|0.0616|2010-09-05 23:00:00|0.0       |0.0     |BTC   |
|0.0616|0.0616|0.0616|0.0616|2010-09-06 00:00:00|0.0       |0.0     |BTC   |
+------+------+------+------+-------------------+----------+--------+------+
only showing top 3 rows



In [6]:
df = df.withColumn("trade_volume", F.abs(df['volumefrom']-df['volumeto']))
df = df.drop('volumefrom', 'volumeto', 'close', 'low', 'open')
df.show(3)

+------+-------------------+------+------------+
|  high|               time|symbol|trade_volume|
+------+-------------------+------+------------+
|0.0616|2010-09-05 22:00:00|   BTC|         0.0|
|0.0616|2010-09-05 23:00:00|   BTC|         0.0|
|0.0616|2010-09-06 00:00:00|   BTC|         0.0|
+------+-------------------+------+------------+
only showing top 3 rows



In [7]:
# new colums for the new df
DELTA_MATCH = {
    'price': {
        'price+24h-avg': 24,
        'price-0h-avg': 0,
        'price-1h-avg': -1,
        'price-2h-avg': -2,
        'price-4h-avg': -4,
        'price-5h-avg': -5,
        'price-6h-avg': -6,
        'price-8h-avg': -8,
        'price-10h-avg': -10,
        'price-12h-avg': -12,
        'price-24h-avg': -24,
        'price-48h-avg': -48,
        'price-96h-avg': -96,
        'price-192h-avg': -192
    },
    'volume': {
        'vol-0h-avg': 0,
        'vol-1h-avg': -1,
        'vol-2h-avg': -2,
        'vol-4h-avg': -4,
        'vol-6h-avg': -6,
        'vol-8h-avg': -8,
        'vol-10h-avg': -10,
        'vol-16h-avg': -16,
        'vol-24h-avg': -24,
        'vol-48h-avg': -48,
        'vol-96h-avg': -96,
        'vol-192h-avg': -192
    }
}

In [8]:
# create and fill columns according to the historic data (in hours timedelta) defined in the PAST_DELTAS dict
symb_window = Window.partitionBy("symbol").orderBy("time")
for symb in CRYPTOS:
    for k, v in DELTA_MATCH['price'].items():
        df = df.withColumn(k, F.lag(df['high'], v*-1).over(symb_window))
    for k, v in DELTA_MATCH['volume'].items():
        df = df.withColumn(k, F.lag(df['trade_volume'], v*-1).over(symb_window))
df.show(2, truncate=False)

+-----+-------------------+------+-----------------+-------------+------------+------------+------------+------------+------------+------------+------------+-------------+-------------+-------------+-------------+-------------+--------------+-----------------+-----------------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+------------+
|high |time               |symbol|trade_volume     |price+24h-avg|price-0h-avg|price-1h-avg|price-2h-avg|price-4h-avg|price-5h-avg|price-6h-avg|price-8h-avg|price-10h-avg|price-12h-avg|price-24h-avg|price-48h-avg|price-96h-avg|price-192h-avg|vol-0h-avg       |vol-1h-avg       |vol-2h-avg|vol-4h-avg|vol-6h-avg|vol-8h-avg|vol-10h-avg|vol-16h-avg|vol-24h-avg|vol-48h-avg|vol-96h-avg|vol-192h-avg|
+-----+-------------------+------+-----------------+-------------+------------+------------+------------+------------+------------+------------+------------+-------------+-------------+-------------+---------

In [9]:
# Remove all the (expected NULL values from the previous operation)
for k, v in DELTA_MATCH.items():
    df = df.filter(df[list(DELTA_MATCH[k].keys())[-1]].isNotNull())

In [10]:
# isolate BTC data (since we don't have an aggregate total market info)
df_btc = df.filter(df['symbol'] == "BTC").cache()

In [11]:
# rename columns in preprapation for future join
df_btc = df_btc.drop('symbol').cache()

In [12]:
for col in df_btc.columns:
    if col == "time": continue
    df_btc = df_btc.withColumnRenamed(col, "btc-{}".format(col)).cache()

In [13]:
# now isolate the non-btc data to prepare for join
df_exbtc = df.filter(df['symbol'] != "BTC").cache()

In [14]:
df_joined = df_exbtc.join(df_btc, 'time', 'inner').cache()

In [16]:
df_joined.toPandas().to_csv('/home/jovyan/data/data_rework_joined_btc.csv')

In [17]:
# Check if the join was correctly executed (same BTC price must appear for all the coins.)
df_joined.select(['high', 'symbol', 'btc-high', 'time']).filter(df['time'] == datetime(2018,1,1,12)).show()

+-------+------+--------+-------------------+
|   high|symbol|btc-high|               time|
+-------+------+--------+-------------------+
|   7.61|   EOS|13343.82|2018-01-01 12:00:00|
| 226.61|   LTC|13343.82|2018-01-01 12:00:00|
| 750.03|   ETH|13343.82|2018-01-01 12:00:00|
|2339.26|   BCH|13343.82|2018-01-01 12:00:00|
| 0.4206|   XLM|13343.82|2018-01-01 12:00:00|
|  0.848|   CVC|13343.82|2018-01-01 12:00:00|
|   1.92|   XRP|13343.82|2018-01-01 12:00:00|
+-------+------+--------+-------------------+



In [18]:
# drop unnecessary/duplicate cols
df_joined = df_joined.drop('high', 'trade_volume', 'btc_high', 'btc-price+24h-avg').cache()

In [19]:
# calculate delta
df_joined = df_joined.withColumn('24h-delta', (df['price+24h-avg'] / df['price-0h-avg']) - 1).cache()

In [21]:
# create a new col with the round delta and eliminate the previous 24h-delta
df_joined = df_joined.withColumn('delta-round', F.round('24h-delta', 2)).drop('24h-delta').cache()
# df_joined.show(1, truncate=False)

In [45]:
# remove NULL from delta-round
df_joined = df_joined.filter(df_joined['delta-round'].isNotNull()).cache()

In [46]:
def categorize_delta(delta) -> int:
    """
    UDF to help categorize growth of a specific crypto currency.
    param: delta - the growth in % we are evaluating
    param: threshold - threshold separating 0 from 1
    """
    if delta is None:
        return 0
    elif delta < 0.04:
        return 0
    else:
        return 1
    
# register the UDF
categorize_delta_udf = F.udf(categorize_delta, IntegerType())
categorize_delta_udf_lambda = F.udf(lambda delta: 1 if delta >= 0.04 else 0, IntegerType())

In [47]:
df_categorized = df_joined.withColumn('category', categorize_delta_udf(df_joined['delta-round'])).cache()

In [48]:
df_categorized.toPandas().to_csv('/home/jovyan/data/data_rework_categorized.csv')